In [2]:
import numpy as np
import pandas as pd
import os
import sys
import feather
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

Tensor are fundamental mathematical construct.
Tensorflow is heavily built on top of Tensor philosophy.
Tensor provide convinient representation of numeric data.

Tensorflow is powerful library for numerical computation. Basic Principle of tensorflow - First Define a python computing graph and tensorflow take that graph and runs it in optimized c ++ code.


Most importantly, it is possible to break up the graph into several chunks and run them in parallel across multiple CPUs or GPUs

## Basics of TensorFLow

### Creating Your First Graph and Running It in a Session

In [3]:
x = tf.Variable(2, name = 'x')
y = tf.Variable(3,name = 'y')
f = x*x*y + y + 2

The above code does not perform any computation, even though it looks like it does. It just creates a computation graph. In fact, even the variables are not initialized yet. To evaluate this graph, you need to open a tensorflow session and use it to initilize the variables and evalute f.

A tensorflow session takes care of placing the operations onto devices such as CPUS and GPUs and running them. Following code completes the computation defined above


In TensorFlow, a tf.Session() object stores the context under which a computation is performed.

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

In [6]:
print(result)

17


In [10]:
sess.close()

Having to repeat sess.run() all the time is a bit cumbersome, but fortunately there is a better way:

In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

17


Here x.initializer.run() is similar to calling tf.get_default_session().run(x.initializer) and similarly f.eval() is similar to calling tf.get_default_session().run(f)

Also, instead of initilizing the variables every single time, we can use folling code to initilize variables globally. Note that it does not actually perform the initialization immediately, but rather creates a node in the graph that will initialize all variables whent it is run:

In [9]:
init = tf.global_variables_initializer() # Prepare an init node

with tf.Session() as sess:
    init.run() # Actully initlaizing all defined variables
    result = f.eval()
    print(result)

17


It is also important close the session after you are done with it. You dont need that with WITH block

Inside Juoyter or within python shell you may prefer to creat an InteractiveSession. The only difference from a regular session is tgat when an InteractiveSession is Created it automatically sets itself as the default session. And Also you have to close the Interactive session

In [11]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

17


In [12]:
sess.close()

A Tensorflow program is typically split into two parts: the first part builds a computation graph. This is called a Construction phase. And second part runs it. It is called an execution phase.

Explicit sessions don’t really show their value until you start to perform computations that have state,

### Basic Tensor Operations

Let's first initialize the interactive gloabal session. And we'll continue to work with it.

In [14]:
sess = tf.InteractiveSession()

TensorFlow provides a number of functions that instantiate basic tensors in memory.

In [16]:
tf.zeros(2)
# rank 1 tensor

<tf.Tensor 'zeros:0' shape=(2,) dtype=float32>

TensorFlow returns a reference to the desired tensor rather than the value of the tensor itself.

In [18]:
a = tf.zeros(2)
a.eval()

array([ 0.,  0.], dtype=float32)

Tensorflow array is built on top of numpy arrys. So, tensor operation would be similar to ndarray operations

In [20]:
a = tf.zeros((2,3)) # Rank 2 Tensor
a.eval()

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

In [22]:
b = tf.ones((2,3,2)) # rank3 tensor
b.eval()

array([[[ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.]],

       [[ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.]]], dtype=float32)

Instead of initializing tensor with one and zeros, we can use tf.fill() to initilize tensor with user input values

In [23]:
b = tf.fill((2,2,2),value=5.)
b.eval()

array([[[ 5.,  5.],
        [ 5.,  5.]],

       [[ 5.,  5.],
        [ 5.,  5.]]], dtype=float32)

In [24]:
a = tf.constant(3) # rank 0 tensor
a.eval()

3

It is much more common to initialize tensors with random values. The most common wat to do this is to sample each entry in the ensor from a random distribution.

In [25]:
a = tf.random_normal((2,2),mean=0, stddev=1)
a.eval()

array([[ 1.34107614,  0.3376978 ],
       [-0.4587625 ,  0.7025398 ]], dtype=float32)

One thing to note is that machine learning systems often make use of very large tensors that often have tens of millions of parameters. When we sample tens of millions of random values from the Normal distribution, it becomes almost certain that some sampled values will be far from the mean. Such large samples can lead to numerical instability, so it’s common to sample using tf.truncated_normal() instead of tf.ran dom_normal(). This function behaves the same as tf.random_normal() in terms of API, but drops and resamples all values more than two standard deviations from the mean.

In [27]:
a = tf.random_uniform((2,2),minval=-2,maxval=2)
a.eval()

array([[ 0.64471245,  0.82815123],
       [ 0.88732767, -0.85938311]], dtype=float32)


Within TensorFlow, tensors are just collections of numbers written in memory. The different shapes are views into the underlying set of numbers that provide different

At different times, it can be useful to view the same set of numbers as forming tensors with different shapes. tf.reshape() allows tensors to be converted into tensors with different shapes

In [28]:
a = tf.ones(8)
a.eval()

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

In [29]:
b = tf.reshape(a,(4,2))
b.eval()

array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.]], dtype=float32)

In [30]:
c = tf.reshape(a,(2,2,2))
c.eval()

array([[[ 1.,  1.],
        [ 1.,  1.]],

       [[ 1.,  1.],
        [ 1.,  1.]]], dtype=float32)

In [32]:
d = tf.reshape(a,(-1,2))
d.eval()

array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.]], dtype=float32)

While all necessary shape manipulations can be performed with tf.reshape(), sometimes it can be convenient to perform simpler shape manipulations using functions such as tf.expand_dims or tf.squeeze. tf.expand_dims adds an extra dimension to a tensor of size 1.

In [33]:
a = tf.ones(2)
a.get_shape()


TensorShape([Dimension(2)])

In [34]:
a.eval()

array([ 1.,  1.], dtype=float32)

In [35]:
b = tf.expand_dims(a,0)
b.get_shape()

TensorShape([Dimension(1), Dimension(2)])

In [36]:
b.eval()

array([[ 1.,  1.]], dtype=float32)

Adding 0 in expand dims creates a row vector

In [37]:
c = tf.expand_dims(a,1)
c.get_shape()

TensorShape([Dimension(2), Dimension(1)])

In [38]:
c.eval()

array([[ 1.],
       [ 1.]], dtype=float32)

Adding 1 in expand dims creates a column Vector

In [39]:
d = tf.squeeze(b)
d.get_shape()

TensorShape([Dimension(2)])

In [40]:
d.eval()

array([ 1.,  1.], dtype=float32)

### Tensorflow graphs

Any computation in Tensorflow is represented as an instance of a tf.Graph object.

Such graph consists of a set of instances of a tf.Tensor objects and tf.Operations objects

When a tf.Graph is not explicitly specified, Tensorflow adds tensors and operations to a hidden global tf.Graph instance

In [41]:
tf.get_default_graph()

### Tensorflow Variables

All the example code in this section has used constant tensors. While we could com‐ bine and recombine these tensors in any way we chose, we could never change the value of tensors themselves (only create new tensors with new values). The

Learning algorithms are essentially rules for updating stored tensors to explain provided data. If it’s not possible to update these stored tensors, it would be hard to learn.

The tf.Variable() class provides a wrapper around tensors that allows for stateful computations. The variable objects serve as holders for tensors. Creating

In [43]:
a = tf.Variable(tf.ones((2, 2)))
a

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32_ref>

In [45]:
#a.eval()

The evaluation fails since variables have to be explicitly initialized. The easiest way to initialize all variables is to invoke tf.global_variables_initializer.

In [46]:
init = tf.global_variables_initializer() # Prepare an init node
init.run()
a.eval()

array([[ 1.,  1.],
       [ 1.,  1.]], dtype=float32)